In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.patches import Rectangle
from PIL import Image
from PIL.ExifTags import TAGS

In [ ]:
def readImage(fullPath, debug=False):
    """
    This methods read an image
    """
    image = cv2.imread(fullPath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    print("Image readed from: " + fullPath)
    if debug:
        type(image)
        image.shape
    return image

def writeImage(fullPath, image, saveQuestion=True,debug= False):
    """
    This method write an image to a path.
    """
    if Path(fullPath).exists() and saveQuestion:
        answer = "-"
        while answer != "y" and answer != "n": 
            answer = input(fullPath +" alreayd exists. Do you want to everride it? (answer y/n)")
            print(answer)
        if answer == "n":
            print("Image not saved")
            return False
    
    
    cv2.imwrite(fullPath, cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    print("Image correctly saved in: " + fullPath)
    return True

def deleteFile(fullPath, validationQuestion=True):
    """
    This method delete a file
    """
    if validationQuestion:
        answer = input("Press enter to continue deleting or any other key to stop it")
        if answer == "":
            os.remove(fullPath)
    else:
        os.remove(fullPath)
    

    
def showImage(image, name=None):
    """
    This method shows the given image
    """
    plt.figure()
    plt.imshow(image)
    #plt.show(block=True)
    if name != None:
        plt.text(0,0,name,fontsize=12,color="red")
    plt.show(block=True)
    
def getImageCrop(image, cropCoords): #croopCoords: x,y,w,h
    """
    This methods returns the given image crop by the coordinates given
    """
    x,y,w,h = cropCoords
    newImage = image[y:y+h, x:x+w]
    return newImage

def showImageBox(image, boxCoords, name=None): #boxCoords: x,y,w,h
    """
    This method shows the given image with the box coords marked.
    """
    plt.figure()
    plt.imshow(image)
    plt.gca().add_patch(Rectangle((boxCoords[0],boxCoords[1]),boxCoords[2],boxCoords[3], edgecolor = "red",
                                  fill=False))
    if name != None:
        plt.text(boxCoords[0],boxCoords[1],name,fontsize=12,color="red")
    plt.show(block=True)
    
def showImageMetadata(fullPath, resultList=[], show=False):
    """
    This method show the specific metadata of an image
    """
    result = {}
    img = Image.open(fullPath)
    info_dict = {
        "Filename": img.filename,
        "Image Size": img.size,
        "Image Height": img.height,
        "Image Width": img.width,
        "Image Format": img.format,
        "Image Mode": img.mode,
        "Image is Animated": getattr(img, "is_animated", False),
        "Frames in Image": getattr(img, "n_frames", 1)
    }
    
    if show:
        for label,value in info_dict.items():
            print(f"{label:25}: {value}")
        
    exifdata = img.getexif()
    if show:
        print("Exif image metadata:", exifdata)
    for tag_id in exifdata:
        
        # get the tag name, instead of human unreadable tag id
        tag = TAGS.get(tag_id, tag_id)
        data = exifdata.get(tag_id)
        if tag in resultList:
            if not isinstance(data,int):
                if "\x00" in data:
                    continue
            result[tag]=data
        # decode bytes 
        try:
            if isinstance(data, bytes):
                data = data.decode()
        except:
            print(info_dict["Filename"], "ERROR IN TAG PROCESSING:", tag)
        if show:
            print(f"{tag:25}: {data}")
    return result
        